데이터 준비
연습 문제를 진행하려면 주식 또는 ETF의 가격 데이터가 필요합니다. Yahoo Finance API 또는 pandas_datareader 등을 이용하여 데이터를 다운로드하거나 가상 데이터를 생성하세요.

In [14]:
import matplotlib.pyplot as plt
import matplotlib as mpl # 한글 폰트 설정 (NanumGothic) 
mpl.rcParams['font.family'] = 'NanumGothic'
mpl.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지

In [15]:
# api 
import FinanceDataReader as fdr
import pandas as pd 

In [16]:
data_df = fdr.DataReader(symbol='AAPL', start='2020')

연습 문제 1: 기본 통계 계산
데이터프레임에 Close(종가) 열이 있다고 가정합니다.
Close 열의 **평균(mean)**과 **중앙값(median)**을 계산하세요.
평균값보다 큰 값의 개수와 중앙값보다 큰 값의 개수를 각각 출력하세요.

In [17]:
data_df['Close'].describe

<bound method NDFrame.describe of 2020-01-02     75.087502
2020-01-03     74.357498
2020-01-06     74.949997
2020-01-07     74.597504
2020-01-08     75.797501
                 ...    
2024-12-18    248.050003
2024-12-19    249.789993
2024-12-20    254.490005
2024-12-23    255.270004
2024-12-24    258.200012
Name: Close, Length: 1254, dtype: float64>

In [18]:
mean = data_df['Close'].mean()
median = data_df['Close'].median()
above_mean = (data_df['Close'] > mean).sum()
above_median = (data_df['Close'] > median).sum()
mean, median, above_mean, above_median

(153.789186590216, 152.5800018310547, 614, 627)

연습 문제 2: 이동 평균(rolling) 계산
Close 열을 기준으로:
7일 이동 평균(rolling mean)을 계산하여 새로운 열로 추가하세요.
7일 이동 평균과 실제 종가의 차이를 계산하여 열로 추가하세요.

In [19]:
data_df['rolling_mean'] = data_df['Close'].rolling(window=7).mean()
data_df['avg-Close'] = data_df['rolling_mean'] - data_df['Close']
data_df['avg-Close']

2020-01-02         NaN
2020-01-03         NaN
2020-01-06         NaN
2020-01-07         NaN
2020-01-08         NaN
                ...   
2024-12-18    0.938570
2024-12-19   -0.512850
2024-12-20   -4.070005
2024-12-23   -3.805719
2024-12-24   -5.297154
Name: avg-Close, Length: 1254, dtype: float64

연습 문제 3: 데이터 이동(shift)
Close 열의 하루 전 가격을 Previous_Close라는 새로운 열로 추가하세요.
Previous_Close와 Close 간의 차이를 계산하여 새로운 열 Daily_Change에 추가하세요.

In [20]:
data_df['Previous_Close'] = data_df['Close'].shift(periods=1)
data_df['Daily_Change'] = data_df['Previous_Close'] - data_df['Close']
data_df['Daily_Change']

2020-01-02         NaN
2020-01-03    0.730003
2020-01-06   -0.592499
2020-01-07    0.352493
2020-01-08   -1.199997
                ...   
2024-12-18    5.429993
2024-12-19   -1.739990
2024-12-20   -4.700012
2024-12-23   -0.779999
2024-12-24   -2.930008
Name: Daily_Change, Length: 1254, dtype: float64

연습 문제 4: 수익과 수익률 최고/최저/중앙값 날짜 확인
수익과 수익률(Daily_Change와 Daily_Change / Previous_Close * 100)을 계산하세요.
수익과 수익률이 가장 높은 날과 가장 낮은 날의 날짜를 출력하세요.
수익과 수익률의 중앙값이 발생한 날짜를 확인하세요.

In [21]:
data_df['Rate_Of_Return'] = (data_df['Daily_Change'] / data_df['Previous_Close']) * 100
data_df['Rate_Of_Return']

2020-01-02         NaN
2020-01-03    0.972204
2020-01-06   -0.796825
2020-01-07    0.470305
2020-01-08   -1.608629
                ...   
2024-12-18    2.142178
2024-12-19   -0.701468
2024-12-20   -1.881585
2024-12-23   -0.306495
2024-12-24   -1.147807
Name: Rate_Of_Return, Length: 1254, dtype: float64

In [22]:
def get_value_index(series, value):
    """주어진 값과 가장 가까운 값의 인덱스를 반환"""
    return (series - value).abs().idxmin()

In [23]:
def get_stats(df, columns, stats):
    """
    df: DataFrame
    columns: list of column names to analyze
    stats: dict of stat name and corresponding callable
    """
    result = {}
    for col in columns:
        result.setdefault(col, {})
        for stat_name, func in stats.items():
            result[col][stat_name] = func(df[col])

    return pd.DataFrame(result).T

In [24]:

columns = ['Rate_Of_Return', 'Close', 'Daily_Change', 'Previous_Close']
columns = ['Rate_Of_Return',  'Daily_Change']
stats = {
    'min': lambda x: x.idxmin(),
    'max': lambda x: x.idxmax(),
    'mean': lambda x: get_value_index(x, x.mean()),
    'median': lambda x: get_value_index(x, x.median())
}
result = get_stats(data_df, columns, stats)
result

,min,max,mean,median
Rate_Of_Return,2020-03-13,2020-03-16,2024-09-27,2024-09-27
Daily_Change,2024-06-11,2024-08-05,2020-08-19,2020-09-15


연습 문제 5: 리샘플링(resample) - 월별 데이터
일별 데이터를 기준으로:
종가(Close)의 월별 평균을 계산하여 출력하세요.
월별 데이터 중 최고값을 기록한 날짜를 확인하세요.

In [25]:
how = {
    'Close': 'mean'
}
resample_df = data_df.resample('MS').apply(how)
resample_df.head()

,Close
2020-01-01,77.979048
2020-02-01,77.817631
2020-03-01,65.611023
2020-04-01,68.096547
2020-05-01,77.496376


In [26]:

columns = ['Close']
stats = {
    'mean': lambda x: get_value_index(x, x.mean())
}
result = get_stats(resample_df, columns, stats)
result

,mean
Close,2021-11-01


연습 문제 6: 52주 고점 및 저점
최근 52주(1년) 동안의 High(최고가)와 Low(최저가)를 각각 계산하여 출력하세요.
고점과 저점의 날짜를 확인하세요.

In [27]:
how = {
    'Close': 'mean'
}
resample_df = data_df.resample('MS').apply(how)
resample_df.head()

,Close
2020-01-01,77.979048
2020-02-01,77.817631
2020-03-01,65.611023
2020-04-01,68.096547
2020-05-01,77.496376


연습 문제 7: 배당 데이터 추가
각 날짜에 가상 배당 데이터를 추가합니다(랜덤 생성 가능).
배당 수익률(배당 / Close * 100)을 계산하여 열로 추가하세요.
가장 높은 배당 수익률을 기록한 날짜를 확인하세요.

연습 문제 8: 데이터 이상치 탐지
Close 열을 기준으로:
이동 평균을 활용하여 이상치(이동 평균 대비 +2 표준편차를 초과하는 값)를 탐지하고 표시하세요.
이상치가 발생한 날짜와 가격을 출력하세요.

연습 문제 9: 최근 5년간 배당률 높은 ETF 선정
KRX 엑셀 활용 :  액티브ETF 실적 - 누적 분배금 참조
최근 5년간 배당률이 높은 ETF 10개를 선정하세요.
선정한 ETF의 배당률과 이름을 출력하세요.

연습 문제 10: 여러 종목 비교
3개의 종목 데이터를 병합하여 각 종목의 Close 데이터만 포함한 데이터프레임을 생성하세요.
각 종목의 수익률을 계산하고 비교하여, 매월 가장 높은 수익률을 기록한 종목과 해당 수익률을 출력하세요.

In [32]:
ticker_name_list = [ # 3 개 KO cocacola, AAPL apple , Alaska Air Group, Inc. (ALK)
    'KO',
    'AAPL',
    'ALK'
]

start_date_str = '2016'
end_date_str = '2019'

# data_df_list = []
data_df = pd.DataFrame()
for symbol in ticker_name_list:
    df = fdr.DataReader(symbol, '2020')
    # data_df_list.append(df)  
    data_df[f'{symbol}'] = df['Close']
    df.info()

# data_df_list[1].head()
data_df.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1254 entries, 2020-01-02 to 2024-12-24
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1254 non-null   float64
 1   High       1254 non-null   float64
 2   Low        1254 non-null   float64
 3   Close      1254 non-null   float64
 4   Volume     1254 non-null   int64  
 5   Adj Close  1254 non-null   float64
dtypes: float64(5), int64(1)
memory usage: 68.6 KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1254 entries, 2020-01-02 to 2024-12-24
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1254 non-null   float64
 1   High       1254 non-null   float64
 2   Low        1254 non-null   float64
 3   Close      1254 non-null   float64
 4   Volume     1254 non-null   int64  
 5   Adj Close  1254 non-null   float64
dtypes: float64(5), int64(1)
memory usage: 68.6 KB
<class 'pa

,KO,AAPL,ALK
2020-01-02,54.990002,75.087502,68.169998
2020-01-03,54.689999,74.357498,66.919998
2020-01-06,54.669998,74.949997,66.599998
2020-01-07,54.250000,74.597504,66.180000
2020-01-08,54.349998,75.797501,66.500000


In [ ]:
for symbol in ticker_name_list:
    data_df[f'{symbol}_Previous_Close'] = data_df[f'{symbol}'].shift(periods=1)
    data_df[f'{symbol}_Daily_Change'] = data_df[f'{symbol}_Previous_Close'] - data_df[f'{symbol}']
    data_df[f'{symbol}_Daily_Change']

In [ ]:
data_df['Rate_Of_Return'] = (data_df['Daily_Change'] / data_df['Previous_Close']) * 100
data_df['Rate_Of_Return']